In [26]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize, rosen
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
#from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, Sum, Product
from sklearn.gaussian_process.kernels import ConstantKernel as C, WhiteKernel as W, RBF, Matern, RationalQuadratic as RQ, ExpSineSquared as ESS, DotProduct as DP, Sum, Product
import warnings
warnings.filterwarnings("ignore")

#===========================================================================================================#
#Obtaining initial values

def conductivity_func(X, finding_min=False):
    if finding_min==False:
        return (rosen(X.T) * 1e-8).round(6)
    elif finding_min==True:
        return -(rosen(X.T) * 1e-8).round(6)

np.random.seed(10003)
samples = 12
concentrations = 4
X = (np.random.random((samples, concentrations))*10).round(3)
y = conductivity_func(X).round(6)

for i in range(len(y)):
    print(f'Sample {i+1}: {X[i]}, {y[i]}')


#===========================================================================================================#
#Declaring ML Model Used

kernel = 31.6**2 * RBF(length_scale=10, length_scale_bounds=(10.0, 10000.0)) + DP(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))
model = GaussianProcessRegressor(kernel = kernel, normalize_y=True, alpha=1e-8, n_restarts_optimizer=3)

#===========================================================================================================#
#Bayesian Optimization

def guess_in_bounds(bounds):
    return [(b[1] - b[0]) * np.random.random() + b[0] for b in bounds]

def acquisition_function1(x, model, explore_weight=1.0):
    pred, std = model.predict([x], return_std=True)
    return -(pred - std * explore_weight)

def acquisition_function2(x, model, explore_weight=1.0):
    pred, std = model.predict([x], return_std=True)
    return -(pred + std * explore_weight)

def acquisition_function3(x, model, best_y, explore_weight=1.0):
    pred, std = model.predict([x], return_std=True)
    return -(norm.cdf((pred - best_y)/std))

bounds = [(0,10)]*concentrations

#===========================================================================================================#
#Global Minimum

model.fit(X, y)

def remove_duplicate(tuple_list):
    for first_index in range(len(tuple_list)-1):
        for second_index in list(range(first_index+1, len(tuple_list))[::-1]):
                if (np.allclose(tuple_list[first_index][0], tuple_list[second_index][0], atol=1e-4)) and (np.isclose(tuple_list[first_index][1], tuple_list[second_index][1], atol=1e-4)):
                        tuple_list.pop(second_index)
    tuple_list = sorted(tuple_list, key=lambda x: x[1], reverse=True)
    return tuple_list

def min_over_bounds(func, bounds, args=None, model_used=False):
    global_mins = []
    iterations = 12
    if model_used == True:
        model = args[0]
    for i in range(iterations):
        np.random.seed(i)
        guess = guess_in_bounds(bounds)
        global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args=args)
        if model_used == False:
            global_mins.append((global_min.x, -global_min.fun))
        else:
            global_mins.append((global_min.x, model.predict(global_min.x.reshape(1, -1))))
    return remove_duplicate(global_mins)

def ML_func(guess, model):
    return -model.predict(guess.reshape(1, -1))

lcb_exploration_weight = 1.0
ucb_exploration_weight = 1.0
poi_exploration_weight= 1.0

mins_to_test = {
    'Conductivity Function': min_over_bounds(conductivity_func, bounds, args=(True)),
    'Non-Baysian Optimization': min_over_bounds(ML_func, bounds, model),
    'Lower Confidence Bound': min_over_bounds(acquisition_function1, bounds, args=(model, lcb_exploration_weight), model_used=True),
    'Upper Confidence Bound': min_over_bounds(acquisition_function2, bounds, args=(model, ucb_exploration_weight), model_used=True),
    'Probability of Improvement': min_over_bounds(acquisition_function3, bounds, args=(model, max(y), poi_exploration_weight), model_used=True)
}

for key in mins_to_test.keys():
    print(f'{key}:')
    for index in range(4):
        print(mins_to_test[key][index][0].round(3), mins_to_test[key][index][1].round(6), conductivity_func(mins_to_test[key][index][0].round(6)))





Sample 1: [9.206 5.806 9.795 7.452], 0.014636
Sample 2: [9.123 5.265 8.88  5.74 ], 0.011781
Sample 3: [0.661 2.222 3.34  7.025], 2.3e-05
Sample 4: [2.022 2.641 2.202 2.139], 3.2e-05
Sample 5: [0.501 2.135 3.23  1.475], 8.6e-05
Sample 6: [0.363 1.749 2.184 1.032], 1.7e-05
Sample 7: [5.306 5.682 0.108 5.365], 0.001569
Sample 8: [0.608 8.135 1.662 2.234], 0.004223
Sample 9: [5.049 3.675 4.728 4.297], 0.000879
Sample 10: [7.569 8.265 5.136 5.122], 0.006847
Sample 11: [0.16  6.248 5.637 4.814], 0.001882
Sample 12: [3.649 6.6   2.837 8.771], 0.001704
Conductivity Function:
[10. 10. 10.  0.] 0.026202 0.026202
[1.e+01 1.e+01 1.e+01 8.e-03] 0.026201 0.026201
[10.  0. 10.  0.] 0.020102 0.020102
[ 0.058 10.    10.     0.   ] 0.018202 0.018202
Non-Baysian Optimization:
[10. 10. 10. 10.] 0.021217 0.024302
[10. 10. 10.  0.] 0.019906 0.026202
[10.  0. 10. 10.] 0.014653 0.018202
[ 0. 10. 10.  0.] 0.009689 0.018202
Lower Confidence Bound:
[10. 10. 10. 10.] [0.021217] 0.024302
[10. 10. 10.  0.] [0.01990

kernel = 31.6**2 * RBF(length_scale=10, length_scale_bounds=(10.0, 10000.0)) + DP(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))

model = GaussianProcessRegressor(kernel = kernel, normalize_y=True, alpha=1e-8)

score = -5.251450383038068e-06

error: -0.0012242895406499746

Conductivity Function:
[10. 10. 10.  0.] 0.026202
[1.e+01 1.e+01 1.e+01 8.e-03] 0.026201
[10.  0. 10.  0.] 0.020102
[ 0.058 10.    10.     0.   ] 0.018202

1.0
Lower Confidence Bound:
[10. 10. 10. 10.] [0.021217] 0.024302
[10. 10. 10.  0.] [0.019906] 0.026202
[10.  0. 10. 10.] [0.014653] 0.018202
[ 0.565 10.     0.352  0.   ] [0.007338] 0.010024
Upper Confidence Bound:
[10. 10. 10. 10.] [0.021217] 0.024302
[10. 10. 10.  0.] [0.019906] 0.026202
[10.  0. 10. 10.] [0.014653] 0.018202
[10.  0. 10.  0.] [0.011726] 0.020102

2.0
Lower Confidence Bound:
[10.    10.    10.     9.508] [0.021007] 0.024391
[10.     9.163 10.    10.   ] [0.020461] 0.021823
[10.     9.092 10.     9.513] [0.020177] 0.021735
[10. 10. 10.  0.] [0.019906] 0.026202
Upper Confidence Bound:
[10. 10. 10.  0.] [0.019906] 0.026202
[10.  0. 10. 10.] [0.014653] 0.018202
[10.  0. 10.  0.] [0.011726] 0.020102
[10. 10.  0.  0.] [0.01108] 0.018102

5.0
Lower Confidence Bound:
[10.     6.555 10.     7.718] [0.017212] 0.018337
[ 9.728  6.348 10.     7.77 ] [0.016555] 0.017222
[0.608 8.136 1.661 2.233] [0.004225] 0.004226
[0.16  6.249 5.637 4.814] [0.001883] 0.001883
Upper Confidence Bound:
[10. 10. 10.  0.] [0.019906] 0.026202
[10.  0. 10. 10.] [0.014653] 0.018202
[10.  0. 10.  0.] [0.011726] 0.020102
[10. 10.  0.  0.] [0.01108] 0.018102

10.0
Lower Confidence Bound:
[10.     6.112 10.     7.196] [0.016645] 0.018178
[ 9.308  5.938 10.     7.713] [0.015432] 0.015671
[7.578 8.27  5.145 5.122] [0.006871] 0.006872
[0.16  6.249 5.637 4.814] [0.001882] 0.001882
Upper Confidence Bound:
[10. 10. 10.  0.] [0.019906] 0.026202
[10.  0. 10. 10.] [0.014653] 0.018202
[10.  0. 10.  0.] [0.011726] 0.020102
[10. 10.  0.  0.] [0.01108] 0.018102